In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
data = pd.read_csv('data.csv')

#pre-processing to calculate outside good shares
shares = data[['Market_ID','Inside Good Share','PPO']].copy()
shares['PPO Share'] = data['Inside Good Share'] * data['PPO']
shares['HMO Share'] = data['Inside Good Share'] * (1 - data['PPO'])

group_shares = shares.groupby('Market_ID').sum()
group_shares['Outside Good Share'] = 1 - group_shares['Inside Good Share']

data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')

print data.min()

Market_ID             1.000000
Plan_ID               1.000000
PPO                   0.000000
Network Score         0.780000
Satisfaction Score    0.725000
Premium               2.269728
Inside Good Share     0.037670
Outside Good Share    0.096070
dtype: float64


In [3]:
def comp_nest_shares(x,name):
    """calculate shares within nest"""
    nest_x = x[['Market_ID',name,'PPO']].copy()
    nest_x['ppo_share'] = nest_x[name] * nest_x['PPO']
    nest_x['hmo_share'] = nest_x[name] * (1 - nest_x['PPO'])
    
    group_shares = nest_x.groupby('Market_ID').sum()
    
    x = pd.merge(x, group_shares[['hmo_share','ppo_share']], right_index=True, left_on = 'Market_ID')
    
    x['nest_size'] =   x['PPO'] * x['ppo_share'] + (1 - x['PPO']) * x['hmo_share']
    x = x.drop(labels=['ppo_share','hmo_share'],axis=1)
    return x


#calculate nest shares
data = comp_nest_shares(data,'Inside Good Share')
data['ln(Within Nest Share)'] = np.log( data['Inside Good Share']/data['nest_size'] )

In [4]:
#first estimate using logit
class logit(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

        
    def momcond(self, params):
        #unwrap stuff
        shares = np.array(self.endog).transpose()
        exog = np.array(self.exog)
        instr = np.array(self.instrument)
        
        lshare = np.log(shares[0]) -  np.log(shares[1])
        lshare = lshare.transpose()
       
        lshare_fit = np.matmul(exog,params) #linear equation    
        
        xi = lshare_fit - lshare
        g = instr * xi[:, np.newaxis]
        
        return g

In [5]:
#calculate hausmann insturments
plan_dum = pd.get_dummies(data['Plan_ID'], prefix='plan',drop_first=True)
mkt_dum = pd.get_dummies(data['Market_ID'], prefix='plan',drop_first=True)
#hausman_instr =  pd.concat( [data[['Network Score','Satisfaction Score','PPO']], mkt_dum],axis=1)

hausman_instr = plan_dum #unsure if this is right...

#set up data for logit
y = data[['Inside Good Share','Outside Good Share']]

# add ln(inside good share) as regressor like formula
x_nested = data[['Network Score','Satisfaction Score','PPO','Premium','ln(Within Nest Share)']]

In [6]:
#set up and run model
beta_nested = np.full(len(x_nested.columns),1)
model = logit(y , x_nested, hausman_instr)
result = model.fit(beta_nested, maxiter=2, optim_method='nm', wargs=dict(centered=False))

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.000006
         Iterations: 571
         Function evaluations: 928
Optimization terminated successfully.
         Current function value: 0.005680
         Iterations: 235
         Function evaluations: 380
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        18.74
Model:                                                   logit   Prob (Hansen J):                0.0437
Method:                                                    GMM                                         
Date:                                         Wed, 10 Oct 2018                                         
Time:                                                 16:10:35                                         
No. Observations:                                         3300                                  

In [11]:
#compute market shares

def comp_shares(x,beta,sigma):
    x = x.copy()
    characs = np.array(x[['Network Score','Satisfaction Score','PPO','Premium']])
    x['exp_delta'] = np.exp(np.matmul(characs,beta)/(1-sigma))
    
    #compute Dg = sum_j|g exp(delta_j)
    shares = x[['Market_ID','exp_delta','PPO']].copy()
    shares['PPO_delta'] = x['exp_delta'] * x['PPO']
    shares['HMO_delta'] = x['exp_delta'] * (1 - x['PPO'])
    
    group_shares =  shares.groupby('Market_ID').sum()
    group_shares['PP0_delta_sigma'] = group_shares['PPO_delta']**(sigma)
    group_shares['HMO_delta_sigma'] = group_shares['HMO_delta']**(sigma)
    group_shares['sum_g'] = group_shares['PPO_delta']**(1-sigma) + group_shares['HMO_delta']**(1-sigma)
    #not sure if there should be a 1 at the end of this... (for outside good?)
    x = pd.merge(x,group_shares[['PPO_delta','HMO_delta','PP0_delta_sigma','sum_g']], 
                right_index=True, left_on = 'Market_ID')

    #compute sum_g Dg^(1-sigma)
    x['denom'] = ( (1 - x['PPO'])*x['HMO_delta']**sigma + x['PPO']*x['PPO_delta']**sigma) * (x['sum_g'])
    x['fitted_share'] = x['exp_delta']/x['denom']
    return x[['Market_ID','Plan_ID','PPO','fitted_share']]

In [12]:
characs = data[['Market_ID','Plan_ID','Network Score','Satisfaction Score','PPO','Premium']]
beta = result.params[:-1]
alpha = abs(beta[3])
sigma = abs(result.params[-1]) #not sure about this, feel like it should be positive?

fitted_shares = comp_shares(characs,beta,sigma)

print fitted_shares['fitted_share'].max(),fitted_shares['fitted_share'].min()
print fitted_shares['fitted_share'].mean(), np.sqrt(fitted_shares['fitted_share'].var())

0.5029674056550822 0.05517143427539584
0.181818181818 0.0792519438840621


In [13]:
#compute total share of each nest
fitted_shares = comp_nest_shares(fitted_shares,'fitted_share')
fitted_shares['nest_shares'] = fitted_shares['fitted_share']/fitted_shares['nest_size']

In [14]:
#aggregate elasticities
fitted_shares['Premium'] = data['Premium']
#can uncomment to get averages
fitted_shares = fitted_shares.groupby('Plan_ID').mean()

#diagonal formula
nest_shares = np.array(fitted_shares['nest_shares'])
shares = np.array(fitted_shares['fitted_share'])
prices = np.array(fitted_shares['Premium'])
ppo = np.array([fitted_shares['PPO']]) #this one is a matrix

#selector matrices
own_price = np.identity(len(shares))
inside_nest = np.matmul(ppo.transpose(),ppo) + np.matmul((1-ppo.transpose()),(1-ppo)) - own_price
outside_nest = 1 - inside_nest - own_price


#elasticity variables
inside_elasticity = np.matmul( np.array( [ (sigma/(1-sigma) * nest_shares + shares)/shares ] ).transpose(),
          np.array([alpha*prices*shares]))

own_elasticity = -alpha*((1/(1-sigma)) - sigma/(1-sigma) * nest_shares - shares)*prices

outside_elasticity =  shares * alpha * prices

nest_elasticity = own_price*own_elasticity + inside_nest*inside_elasticity + outside_nest*outside_elasticity


print outside_nest
print nest_elasticity[1][1]
print nest_elasticity[0][1],nest_elasticity[2][1] #should be the same (outside nest)
print nest_elasticity[4][1],nest_elasticity[5][1] #should be different

[[0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]]
-3.565235846680758
0.3439468873809879 0.3439468873809879
0.9450527859032544 0.9123970611579443
